<a href="https://colab.research.google.com/github/mh-amani/twitter-sentiment-classification/blob/main/masani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## installing libraries, loading the dataset, train/val split

In [ ]:
#@title installing dependecies
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 12.9 MB/s 
     |████████████████████████████████| 7.6 MB 44.6 MB/s 
     |████████████████████████████████| 182 kB 66.6 MB/s 


In [ ]:
#@title mount your Google Drive
#@markdown progress will be saved in the current folder in google-drive 

import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title set up mount symlink

# path on google drive to save your files, data 
# The notebook should also be in same directory
DRIVE_PATH = '/content/gdrive/My\ Drive/Colab\ Notebooks/twitter-sentiment'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/twitter-sentiment'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [ ]:
#@title clone dataset + codes from github
%cd $SYM_PATH
DATA_PATH = SYM_PATH + '/twitter-sentiment-classification'
if not os.path.exists(DATA_PATH):
  !git clone https://github.com/mh-amani/twitter-sentiment-classification


%cd $DATA_PATH
DATA_PATH = DATA_PATH + '/twitter-datasets'
if not os.path.exists(DATA_PATH):
  !unzip 'twitter-datasets'
%cd '../'
%pwd

/content/gdrive/My Drive/Colab Notebooks/twitter-sentiment
/content/gdrive/My Drive/Colab Notebooks/twitter-sentiment/twitter-sentiment-classification
/content/gdrive/My Drive/Colab Notebooks/twitter-sentiment


'/content/gdrive/My Drive/Colab Notebooks/twitter-sentiment'

In [ ]:
#@title read dataset files
from os import listdir
from os.path import isfile, join

Dataset = {}

filenames = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]
print(filenames)
for filename in filenames:
  with open(DATA_PATH + '/' + filename, 'r') as f:
    lines = f.read().splitlines() 
    Dataset[filename[:-4]] = lines

print(Dataset.keys())

['train_pos_full.txt', 'train_neg_full.txt', 'test_data.txt', 'train_pos.txt', 'train_neg.txt', 'sample_submission.csv']
dict_keys(['train_pos_full', 'train_neg_full', 'test_data', 'train_pos', 'train_neg', 'sample_submission'])


In [ ]:
import torch
import numpy as np
from pprint import pprint
from sklearn.model_selection import train_test_split


train_text_pos = Dataset['train_pos']
train_text_neg = Dataset['train_neg']
test_text = Dataset['test_data']
train_text = train_text_pos + train_text_neg
train_label = np.ones((len(train_text_pos) + len(train_text_neg)))
train_label[len(train_text_pos):] = train_label[len(train_text_pos):] * -1

train_texts, val_texts, train_labels, val_labels = train_test_split(train_text, train_label, test_size=.1)


pprint(train_texts[:5])
print()
print(train_texts[0])

["<user> my naviator , ahahah ! i dare you ! you'd have to pay then i would "
 'get a 2012 one',
 '<user> good morning , divine ! please follow <user> carlo has followed us na '
 '! sana ikaw din . thanks ! loveya ! ) ) )',
 'last day of spring classes today',
 '<user> your brother keeps calling me boo ! please tell him to stop',
 "rt <user> i'll eat u up if i have 2 ... jst want u 2 b wet as hell ma ! ! !"]

<user> my naviator , ahahah ! i dare you ! you'd have to pay then i would get a 2012 one


## preprocessings

## using BERT

### tokenization and making the dataset

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_labels[train_labels==-1] = 0
val_labels[val_labels == -1] = 0
train_labels = train_labels.reshape((-1, 1))
val_labels = val_labels.reshape((-1, 1))

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

class make_torch_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=[]):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels):
            item["labels"] = torch.tensor(self.labels[idx]).to(torch.int64)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = make_torch_dataset(train_encodings, train_labels)
val_dataset = make_torch_dataset(val_encodings, val_labels)
test_dataset = make_torch_dataset(test_encodings)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropo

In [ ]:
id = 100
print(train_dataset[id])
print(train_encodings[id].ids)
print(train_encodings[id].tokens)
print(train_encodings[id].offsets)
print(len(train_encodings[id].ids))
print(test_dataset[id])

{'input_ids': tensor([  101,  1045,  3403,  2006,  1996,  2154,  1026,  5310,  1028,  2097,
         1056, 28394,  2102,  2033,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 ### Training


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(p):    
    pred, labels = p
    print(p)
    print(pred)
    print(labels)
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    eval_steps=1000,
    evaluation_strategy="steps",
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=256,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    save_steps=4000,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=20,
    seed=0,
    load_best_model_at_end=True,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    compute_metrics=compute_metrics,
    eval_dataset=val_dataset             # evaluation dataset
)

# Train pre-trained model
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights fi

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1000,0.351600,0.353460,0.840300,0.854175,0.818621,0.836020
2000,0.359100,0.324970,0.853650,0.857347,0.846571,0.851925
3000,0.299600,0.314708,0.859500,0.823188,0.913734,0.866101
4000,0.309200,0.300138,0.866750,0.850351,0.888397,0.868958
5000,0.300300,0.308869,0.864650,0.890579,0.829781,0.859106
6000,0.197600,0.320377,0.871250,0.865299,0.877740,0.871475
7000,0.239000,0.308432,0.869700,0.869215,0.868691,0.868953
8000,0.236700,0.303319,0.873200,0.867778,0.878946,0.873327
9000,0.224600,0.292885,0.874300,0.850434,0.906696,0.877664
10000,0.244300,0.303131,0.874900,0.872573,0.876433,0.874498


***** Running Evaluation *****
  Num examples = 20000
  Batch size = 256


[[-1.9955037  1.825776 ]
 [ 3.3767016 -3.7145348]
 [-2.2709436  2.0889757]
 ...
 [ 3.3571825 -3.6847613]
 [-1.8852562  1.6599241]
 [ 3.371807  -3.7039704]]
[[1]
 [0]
 [1]
 ...
 [0]
 [1]
 [0]]


***** Running Evaluation *****
  Num examples = 20000
  Batch size = 256


...

[[-2.8702304  2.4428856]
 [ 4.979283  -5.629801 ]
 [-3.4038515  3.0625663]
 ...
 [ 4.8869195 -5.551708 ]
 [-2.5212567  2.127591 ]
 [ 4.918385  -5.5893445]]
[[1]
 [0]
 [1]
 ...
 [0]
 [1]
 [0]]


***** Running Evaluation *****
  Num examples = 20000
  Batch size = 256


[[-3.025743   2.5798542]
 [ 5.0347247 -5.70689  ]
 [-3.530765   3.1915054]
 ...
 [ 4.9623585 -5.654513 ]
 [-2.8239794  2.3923635]
 [ 4.960134  -5.6645947]]
[[1]
 [0]
 [1]
 ...
 [0]
 [1]
 [0]]


Saving model checkpoint to ./results/checkpoint-16000
Configuration saved in ./results/checkpoint-16000/config.json
Model weights saved in ./results/checkpoint-16000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-4000 (score: 0.3001382350921631).


TrainOutput(global_step=16875, training_loss=0.23140729125693993, metrics={'train_runtime': 5051.3432, 'train_samples_per_second': 106.902, 'train_steps_per_second': 3.341, 'total_flos': 1.439030608056e+16, 'train_loss': 0.23140729125693993, 'epoch': 3.0})

In [ ]:
from datetime import datetime
print('saving model_{}'.format(datetime.now().strftime('%Y-%m-%d-%H-%M')))
model_address = './model_{}'.format(datetime.now().strftime('%Y-%m-%d-%H-%M'))
trainer.save_model(model_address)

Saving model checkpoint to ./model_2022-12-04-14-11
Configuration saved in ./model_2022-12-04-14-11/config.json


saving model_2022-12-04-14-11


Model weights saved in ./model_2022-12-04-14-11/pytorch_model.bin


In [ ]:
model.eval()

# Create torch dataset
# test_dataset = make_torch_dataset(X_test_tokenized) 

# Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2) 

# Define test trainer
test_trainer = Trainer(model) 

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset) 

# Preprocess raw predictions
y_pred = (np.argmax(raw_pred, axis=1)).reshape(-1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


In [ ]:
# Define test trainer
test_trainer = Trainer(model) 

# Make prediction
raw_pred, _, _ = test_trainer.predict(val_dataset) 

# Preprocess raw predictions
y_pred = (np.argmax(raw_pred, axis=1)).reshape(-1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 20000
  Batch size = 8


In [ ]:
print(y_pred.mean())

0.529


In [ ]:
print(val_labels)

[[1.]
 [0.]
 [1.]
 ...
 [0.]
 [1.]
 [0.]]


In [ ]:
y_pred[y_pred==0]=-1

In [ ]:
print(y_pred.mean())

0.058


In [ ]:
print(test_dataset[0])

{'input_ids': tensor([  101,  1015,  1010,  2712, 20160,  4013,  2712,  8040, 17206,  2121,
         1006,  2998,  2007,  1996, 12109,  2712,  1011, 20160, 11915,  3597,
        12184, 22381,  3726,  2250,  1010,  2994,  2936,  1999,  1996,  2300,
         1998,  1012,  1012,  1012,  1026, 24471,  2140,  1028,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


In [ ]:
# ## checkinbg models output-input form:
# jj = train_dataset[0:1]
# del(jj['labels'])
# print(jj)
# print(model(**jj))

0.8685

In [ ]:
raw_pred

array([[ 3.9577732 , -4.421872  ],
       [ 0.49447355, -0.78333306],
       [ 0.5419452 , -0.7995021 ],
       ...,
       [ 3.9494116 , -4.372576  ],
       [-1.8046775 ,  1.4894595 ],
       [ 4.1487284 , -4.6502647 ]], dtype=float32)

In [ ]:
y_pred = (np.argmax(raw_pred, axis=1)).reshape(-1)

## saving output csv file from predictions

In [ ]:
y_pred = y_pred.astype(int)
print(y_pred)
y_pred[y_pred==0] = -1


[-1 -1 -1 ... -1  1 -1]
0.058


In [ ]:
import pandas as pd
labeling_df = pd.read_csv(DATA_PATH+'/'+'sample_submission.csv')
labeling_df['Prediction'] = y_pred
labeling_df.to_csv('pred1.csv')

In [ ]:
labeling_df

,Id,Prediction
0,1,-1
1,2,-1
2,3,-1
3,4,1
4,5,-1
...,...,...
9995,9996,1
9996,9997,-1
9997,9998,-1
9998,9999,1


## decrepted code (trash)

In [ ]:
# from torch.utils.data import DataLoader
# from transformers import DistilBertForSequenceClassification, AdamW

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()